In [209]:
# Load Libraries:- 
import seaborn as sns 
import matplotlib.pyplot as plt 
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer , CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


In [4]:
# !pip install nltk

## Resume Screening Using NLP

## Resume Dataset:-

In [200]:
df = pd.read_csv('ML/UpdatedResumeDataSet.csv')
df.head()

,Category,Resume
0,Data Science,Skills * Programming Languages: Python (pandas...
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...
2,Data Science,"Areas of Interest Deep Learning, Control Syste..."
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab..."


# Jobs Descriptions:- 

In [201]:
jobs_df = pd.read_csv('ML/job_descriptions.csv')
jobs_df.head()

,Job Id,Experience,Qualifications,Salary Range,location,Country,latitude,longitude,Work Type,Company Size,...,Contact,Job Title,Role,Job Portal,Job Description,Benefits,skills,Responsibilities,Company,Company Profile
0,1089843540111562,5 to 15 Years,M.Tech,$59K-$99K,Douglas,Isle of Man,54.2361,-4.5481,Intern,26801,...,001-381-930-7517x737,Digital Marketing Specialist,Social Media Manager,Snagajob,Social Media Managers oversee an organizations...,"{'Flexible Spending Accounts (FSAs), Relocatio...","Social media platforms (e.g., Facebook, Twitte...","Manage and grow social media accounts, create ...",Icahn Enterprises,"{""Sector"":""Diversified"",""Industry"":""Diversifie..."
1,398454096642776,2 to 12 Years,BCA,$56K-$116K,Ashgabat,Turkmenistan,38.9697,59.5563,Intern,100340,...,461-509-4216,Web Developer,Frontend Web Developer,Idealist,Frontend Web Developers design and implement u...,"{'Health Insurance, Retirement Plans, Paid Tim...","HTML, CSS, JavaScript Frontend frameworks (e.g...","Design and code user interfaces for websites, ...",PNC Financial Services Group,"{""Sector"":""Financial Services"",""Industry"":""Com..."
2,481640072963533,0 to 12 Years,PhD,$61K-$104K,Macao,"Macao SAR, China",22.1987,113.5439,Temporary,84525,...,9687619505,Operations Manager,Quality Control Manager,Jobs2Careers,Quality Control Managers establish and enforce...,"{'Legal Assistance, Bonuses and Incentive Prog...",Quality control processes and methodologies St...,Establish and enforce quality control standard...,United Services Automobile Assn.,"{""Sector"":""Insurance"",""Industry"":""Insurance: P..."
3,688192671473044,4 to 11 Years,PhD,$65K-$91K,Porto-Novo,Benin,9.3077,2.3158,Full-Time,129896,...,+1-820-643-5431x47576,Network Engineer,Wireless Network Engineer,FlexJobs,"Wireless Network Engineers design, implement, ...","{'Transportation Benefits, Professional Develo...",Wireless network design and architecture Wi-Fi...,"Design, configure, and optimize wireless netwo...",Hess,"{""Sector"":""Energy"",""Industry"":""Mining, Crude-O..."
4,117057806156508,1 to 12 Years,MBA,$64K-$87K,Santiago,Chile,-35.6751,-71.5429,Intern,53944,...,343.975.4702x9340,Event Manager,Conference Manager,Jobs2Careers,A Conference Manager coordinates and manages c...,"{'Flexible Spending Accounts (FSAs), Relocatio...",Event planning Conference logistics Budget man...,Specialize in conference and convention planni...,Cairn Energy,"{""Sector"":""Energy"",""Industry"":""Energy - Oil & ..."


In [64]:
jobs_df['Job Description'][0]

'Social Media Managers oversee an organizations social media presence. They create and schedule content, engage with followers, and analyze social media metrics to drive brand awareness and engagement.'

In [23]:
# Preprocess text data: remove punctuation, stopwords, etc.
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
def preprocess_text(text):
    text = re.sub(r'\W', ' ', text)  # Remove special characters
    text = text.lower()
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text

In [17]:
# Function to handle lowercase conversion safely 
def to_lower(x): 
    if isinstance(x, str): 
        return x.lower() 
    return x

In [24]:
def preprocess_text1(text): 
    if not isinstance(text, str): 
        return ''
    text = to_lower(text)
    tokens = word_tokenize(text) 
    tokens = [word for word in tokens if word.isalpha()] # Remove punctuation 
    tokens = [word for word in tokens if word not in stop_words] # Remove stopwords 
    return ' '.join(tokens)

# Preprocessing or cleaning text:-

In [25]:
df['processed_text'] = df['Resume'].apply(preprocess_text)
# df['processed_text1'] = df['Resume'].apply(preprocess_text1)

In [65]:
jobs_df['processes_test'] = jobs_df['Job Description'].apply(preprocess_text)
#jobs_df['processes_test1'] = jobs_df['Job Description'].apply(preprocess_text1)

In [26]:
df.head()

,Category,Resume,processed_text
0,Data Science,Skills * Programming Languages: Python (pandas...,skills programming languages python pandas num...
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...,education details may 2013 may 2017 b e uit rg...
2,Data Science,"Areas of Interest Deep Learning, Control Syste...",areas interest deep learning control system de...
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...,skills â r â python â sap hana â tableau â sap...
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab...",education details mca ymcaust faridabad haryan...


In [66]:
jobs_df.head()

,Job Id,Experience,Qualifications,Salary Range,location,Country,latitude,longitude,Work Type,Company Size,...,Job Title,Role,Job Portal,Job Description,Benefits,skills,Responsibilities,Company,Company Profile,processes_test
0,1089843540111562,5 to 15 Years,M.Tech,$59K-$99K,Douglas,Isle of Man,54.2361,-4.5481,Intern,26801,...,Digital Marketing Specialist,Social Media Manager,Snagajob,Social Media Managers oversee an organizations...,"{'Flexible Spending Accounts (FSAs), Relocatio...","Social media platforms (e.g., Facebook, Twitte...","Manage and grow social media accounts, create ...",Icahn Enterprises,"{""Sector"":""Diversified"",""Industry"":""Diversifie...",social media managers oversee organizations so...
1,398454096642776,2 to 12 Years,BCA,$56K-$116K,Ashgabat,Turkmenistan,38.9697,59.5563,Intern,100340,...,Web Developer,Frontend Web Developer,Idealist,Frontend Web Developers design and implement u...,"{'Health Insurance, Retirement Plans, Paid Tim...","HTML, CSS, JavaScript Frontend frameworks (e.g...","Design and code user interfaces for websites, ...",PNC Financial Services Group,"{""Sector"":""Financial Services"",""Industry"":""Com...",frontend web developers design implement user ...
2,481640072963533,0 to 12 Years,PhD,$61K-$104K,Macao,"Macao SAR, China",22.1987,113.5439,Temporary,84525,...,Operations Manager,Quality Control Manager,Jobs2Careers,Quality Control Managers establish and enforce...,"{'Legal Assistance, Bonuses and Incentive Prog...",Quality control processes and methodologies St...,Establish and enforce quality control standard...,United Services Automobile Assn.,"{""Sector"":""Insurance"",""Industry"":""Insurance: P...",quality control managers establish enforce qua...
3,688192671473044,4 to 11 Years,PhD,$65K-$91K,Porto-Novo,Benin,9.3077,2.3158,Full-Time,129896,...,Network Engineer,Wireless Network Engineer,FlexJobs,"Wireless Network Engineers design, implement, ...","{'Transportation Benefits, Professional Develo...",Wireless network design and architecture Wi-Fi...,"Design, configure, and optimize wireless netwo...",Hess,"{""Sector"":""Energy"",""Industry"":""Mining, Crude-O...",wireless network engineers design implement ma...
4,117057806156508,1 to 12 Years,MBA,$64K-$87K,Santiago,Chile,-35.6751,-71.5429,Intern,53944,...,Event Manager,Conference Manager,Jobs2Careers,A Conference Manager coordinates and manages c...,"{'Flexible Spending Accounts (FSAs), Relocatio...",Event planning Conference logistics Budget man...,Specialize in conference and convention planni...,Cairn Energy,"{""Sector"":""Energy"",""Industry"":""Energy - Oil & ...",conference manager coordinates manages confere...


In [144]:
jobs_df['Role'].unique()

array(['Social Media Manager', 'Frontend Web Developer',
       'Quality Control Manager', 'Wireless Network Engineer',
       'Conference Manager', 'Quality Assurance Analyst',
       'Classroom Teacher', 'User Interface Designer',
       'Interaction Designer', 'Wedding Consultant',
       'Performance Testing Specialist', 'Family Law Attorney',
       'Mechanical Design Engineer', 'Network Security Analyst',
       'Sales Account Manager', 'Product Brand Manager',
       'School Social Worker', 'Content Creator',
       'Deliverability Analyst', 'HR Coordinator', 'Legal Secretary',
       'Family Nurse Practitioner', 'Account Strategist',
       'Backend Developer', 'Supply Chain Coordinator',
       'B2B Sales Consultant', 'Structural Engineer',
       'Security Operations Center (SOC) Analyst', 'Front-End Developer',
       'Tax Planner', 'Event Coordinator', 'Clinical Psychologist',
       'Electrical Engineer', 'Lighting Designer',
       'Business Intelligence Analyst', 'Conten

In [143]:
jobs_df['Job Description'].unique()

array(['Social Media Managers oversee an organizations social media presence. They create and schedule content, engage with followers, and analyze social media metrics to drive brand awareness and engagement.',
       'Frontend Web Developers design and implement user interfaces for websites, ensuring they are visually appealing and user-friendly. They collaborate with designers and backend developers to create seamless web experiences for users.',
       'Quality Control Managers establish and enforce quality standards within an organization. They develop quality control processes, perform inspections, and implement corrective actions to maintain product or service quality.',
       'Wireless Network Engineers design, implement, and maintain wireless network solutions. They optimize wireless connectivity, troubleshoot issues, and ensure reliable and secure wireless communications.',
       'A Conference Manager coordinates and manages conferences, meetings, and events. They plan logis

In [142]:
jobs_df['Job Description'].value_counts()

Job Description
Interaction Designers specialize in designing user interactions within digital interfaces. They create meaningful and engaging user experiences by considering user behaviors and system responses.                                                                            20580
Network Administrators manage an organizations computer networks. They configure, monitor, and maintain network infrastructure, ensuring optimal performance and security.                                                                                                     17470
User Interface Designers focus on the visual and interactive aspects of digital interfaces. They design layouts, buttons, and other elements to ensure a cohesive and visually appealing user interface.                                                                       14036
Social Media Managers oversee an organizations social media presence. They create and schedule content, engage with followers, and analyze social media m

In [128]:
data = {"title":["Data Science Director"],
"company":["Altoida"],
"job_description":["Altoida is a pioneer in developing digital biomarkers of neurological disease using augmented reality and machine learning. Our technology platform is designed to enable an objective evaluation of an individual’s neurological health, allowing for more accurate patient selection and stratification for clinical trials, as well as sensitive monitoring of disease progression. Altoida’s mission is to provide life sciences companies with actionable insights using real-world data to increase the success rate for novel therapeutics, and usher in a new era of precision neurology using digital biomarkers. Our proprietary evidence-based platform is founded on more than 20 years of scientific research and published in multiple peer-reviewed papers including Nature Digital Medicine. For more information, visit www.altoida.com. Follow us on Twitter @altoida.  Altoida’s culture is united around six core principles: - Patient-First - Open Communication - Collaborative  - Reliable - Embrace Positive Change - Think and Build at Scale We are looking for an entrepreneurial Data Science leader who values moving quickly to innovate and execute product initiatives. The Director of Data Science helps develop product strategies and lays out the analytic vision and roadmap to help the integrated Product Development team to deliver accordingly. This leader will bring robust, end-to-end analytics leadership experience that includes data science life cycle processes and data engineering, and work closely with other leaders, including Product Management and Engineering, to establish a world-class culture of data analytics and machine learning. They will report to the Vice President of Product and Software Development. Responsibilities (Leadership) Influence leadership and peers across functional lines within Altoida to develop a consistent and transparent digital and data science strategy  Build and maintain partnerships with internal and external stakeholders to augment data science capability and innovation Responsibilities (Data Science) Demonstrated experience and understanding of generative, discriminative, and contrastive ML methods with application in classification and gradient boosting techniques, knowledge discovery and causal inference  Broad understanding of applied mathematics and statistics (e.g., Bayesian, Multi-Objective Optimization, Topological Data Analysis) and effective application in addressing multiple comparison problems, data independence, collinearity and other factors that improve overall interpretation of results  Willing to take a hands-on role to deliver on multiple, highly visible projects, plus the ability to develop and coach others, and guide other team members to achieve outcomes that create meaningful business impact  Serve as an ambassador for Data Science at Altoida by presenting and publishing articles Build a bridge between academia, technology, and industry partners to drive improvements in data science capability & processes, including access to external data from regulatory and other 3rd party sources, as applicable  Partner internally and cross-functionally on data-driven strategies to ensure an optimized and consistent approach (data standards) to metrics, thresholds, etc. that meet stakeholder needs  Assess, influence and ensure data quality / validation procedures, including managing the curation, documentation, protection, and storage as per policy  Explore / coordinate implementation advancements in AI, ML and related technology / tools, including external benchmarking to pharma and other leading organizations  Strong ability to explain ML and AI concepts internally and to regulators, as applicable to our products, including how to minimize bias in the product and increase scale  Guide the continuous calibration of model outputs to maximize performance and external validity, while following data security protocols  Provide objective assessments to assist leadership in prioritizing investments in data-science capabilities and projects  Keep pace with latest developments in the field and mentor associates accordingly Responsibilities (Operational Excellence) Accountable for design and timely delivery of the digital / data science strategies  Accountable for ensuring validation / controls are in place to support data science capabilities and models, including the consistent application of data quality standards  Accountable for delivering on ML research & development across multiple concurrent projects and smooth transition to production, as applicable Desired Skills and Experiences 10+ years of work experience leading data teams, working collaboratively with Product, Engineering and cross-functional partners, and guiding data-driven product planning, prioritization, and strategy development  Executive-level communication skills for cross-functional collaboration across teams  Able to think innovatively and to develop and execute strategic plans pursuant to the responsibilities of the role, solving complex and unique problems  Previous people leadership experience and experience leading the development and technical growth of colleagues to support business transformation  Experience in hiring, retaining, and scaling geographically dispersed, high-performing teams Experience influencing product strategy and driving change across organizational boundaries through clear and compelling communication of data-driven insights and analyses  Experience in a technical leadership role, such as Statistics, Mathematics, Computer Science, or a related field with strong analytical skills to assess information, draw conclusions and recommend improvements  Well-developed interpersonal skills with ability to collaborate across functions and work effectively across all levels of the organization  Knowledge of Quality Management Systems, healthcare data standards and regulations, including HIPAA  Experience with Python object-oriented development and related scientific & statistical packages, SQL, and other programming languages The position is remote with Greater Boston, MA candidates preferred. Minor travel is expected. Compensation Altoida provides competitive and comprehensive compensation and benefits programs. Specific benefit offerings may vary by location, position, and/or business unit. The full-time salary range is commensurate with experience. Location Altoida’s US headquarters is located in Washington, DC. The position is remote. Equal Opportunity Altoida does not discriminate based upon race, religion, color, national origin, sex (including pregnancy, childbirth, or related medical conditions), sexual orientation, gender, age, status as a protected veteran, status as an individual with a disability, or other applicable legally protected characteristics. Altoida is committed to providing reasonable accommodations for candidates with disabilities in our recruiting process."
                  ]}
new_des = pd.DataFrame(data)
new_data = new_des['job_description'].apply(preprocess_text)
new_data[0]
#new_des.head()

'altoida pioneer developing digital biomarkers neurological disease using augmented reality machine learning technology platform designed enable objective evaluation individual neurological health allowing accurate patient selection stratification clinical trials well sensitive monitoring disease progression altoida mission provide life sciences companies actionable insights using real world data increase success rate novel therapeutics usher new era precision neurology using digital biomarkers proprietary evidence based platform founded 20 years scientific research published multiple peer reviewed papers including nature digital medicine information visit www altoida com follow us twitter altoida altoida culture united around six core principles patient first open communication collaborative reliable embrace positive change think build scale looking entrepreneurial data science leader values moving quickly innovate execute product initiatives director data science helps develop produc

In [29]:
df['Resume'][0]

'Skills * Programming Languages: Python (pandas, numpy, scipy, scikit-learn, matplotlib), Sql, Java, JavaScript/JQuery. * Machine learning: Regression, SVM, NaÃ¯ve Bayes, KNN, Random Forest, Decision Trees, Boosting techniques, Cluster Analysis, Word Embedding, Sentiment Analysis, Natural Language processing, Dimensionality reduction, Topic Modelling (LDA, NMF), PCA & Neural Nets. * Database Visualizations: Mysql, SqlServer, Cassandra, Hbase, ElasticSearch D3.js, DC.js, Plotly, kibana, matplotlib, ggplot, Tableau. * Others: Regular Expression, HTML, CSS, Angular 6, Logstash, Kafka, Python Flask, Git, Docker, computer vision - Open CV and understanding of Deep learning.Education Details \r\n\r\nData Science Assurance Associate \r\n\r\nData Science Assurance Associate - Ernst & Young LLP\r\nSkill Details \r\nJAVASCRIPT- Exprience - 24 months\r\njQuery- Exprience - 24 months\r\nPython- Exprience - 24 monthsCompany Details \r\ncompany - Ernst & Young LLP\r\ndescription - Fraud Investigatio

In [30]:
df['processed_text'][0]

'skills programming languages python pandas numpy scipy scikit learn matplotlib sql java javascript jquery machine learning regression svm naã bayes knn random forest decision trees boosting techniques cluster analysis word embedding sentiment analysis natural language processing dimensionality reduction topic modelling lda nmf pca neural nets database visualizations mysql sqlserver cassandra hbase elasticsearch d3 js dc js plotly kibana matplotlib ggplot tableau others regular expression html css angular 6 logstash kafka python flask git docker computer vision open cv understanding deep learning education details data science assurance associate data science assurance associate ernst young llp skill details javascript exprience 24 months jquery exprience 24 months python exprience 24 monthscompany details company ernst young llp description fraud investigations dispute services assurance technology assisted review tar technology assisted review assists accelerating review process run 

In [31]:
df['processed_text1'][0]

'skills programming languages python pandas numpy scipy matplotlib sql java machine learning regression svm bayes knn random forest decision trees boosting techniques cluster analysis word embedding sentiment analysis natural language processing dimensionality reduction topic modelling lda nmf pca neural nets database visualizations mysql sqlserver cassandra hbase elasticsearch plotly kibana matplotlib ggplot tableau others regular expression html css angular logstash kafka python flask git docker computer vision open cv understanding deep details data science assurance associate data science assurance associate ernst young llp skill details exprience months exprience months exprience monthscompany details company ernst young llp description fraud investigations dispute services assurance technology assisted review tar technology assisted review assists accelerating review process run analytics generate reports core member team helped developing automated review platform tool scratch a

# Extract features using bag-of-words

In [ ]:
# Extract features using bag-of-words
#vectorizer = CountVectorizer()
#vectorizer = TfidfVectorizer()

In [68]:
jobs_df.columns

Index(['Job Id', 'Experience', 'Qualifications', 'Salary Range', 'location',
       'Country', 'latitude', 'longitude', 'Work Type', 'Company Size',
       'Job Posting Date', 'Preference', 'Contact Person', 'Contact',
       'Job Title', 'Role', 'Job Portal', 'Job Description', 'Benefits',
       'skills', 'Responsibilities', 'Company', 'Company Profile',
       'processes_test'],
      dtype='object')

In [101]:
# Convert resume and job description text to TF-IDF vectors
vectorizer = TfidfVectorizer(max_features=400)
resume_vectors = vectorizer.fit_transform(df['processed_text'])
job_vectors = vectorizer.transform(jobs_df['processes_test'])


In [102]:
# Calculate similarity score (using cosine similarity)
from sklearn.metrics.pairwise import cosine_similarity

In [103]:
resume_vectors.toarray()[0]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.1297055 , 0.        , 0.        , 0.13464949,
       0.        , 0.        , 0.        , 0.        , 0.03973061,
       0.28231088, 0.        , 0.31464802, 0.04213305, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.03111758, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.05473382, 0.        ,
       0.08999961, 0.        , 0.        , 0.0515057 , 0.04323517,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.07489974, 0.05519145, 0.05691407,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.01487171, 0.        , 0.        , 0.02971272,
       0.        , 0.        , 0.        , 0.        , 0.03908

In [104]:
job_vectors.toarray()[0]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [133]:
new_data_vectors = vectorizer.transform(new_data)
new_data_vectors.toarray()[0]

array([0.02895058, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.10179384, 0.03860068, 0.20463894,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.02681583, 0.        , 0.07969981, 0.        , 0.        ,
       0.06811981, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.04729219, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.10258538, 0.        , 0.07031428, 0.        , 0.        ,
       0.        , 0.07043791, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.03860068, 0.        ,
       0.08426742, 0.        , 0.        , 0.        , 0.02257855,
       0.        , 0.        , 0.        , 0.        , 0.02970

In [ ]:
new_data_vectors = new_data_vectors[0]

In [134]:
# Assume we are matching resumes with a specific job
job_vector = job_vectors[1]  # Example for the first job description
similarity_scores = cosine_similarity(resume_vectors, new_data_vectors)

In [135]:
# Add similarity scores to resume data
df['similarity_score'] = similarity_scores

In [138]:
df.head(50)
# df.iloc[200:300,:].head(50)
# df.iloc[500:600,:].head(50)

,Category,Resume,processed_text,processed_text1,similarity_score
0,Data Science,Skills * Programming Languages: Python (pandas...,skills programming languages python pandas num...,skills programming languages python pandas num...,0.401829
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...,education details may 2013 may 2017 b e uit rg...,education details may may data scientist data ...,0.147394
2,Data Science,"Areas of Interest Deep Learning, Control Syste...",areas interest deep learning control system de...,areas interest deep learning control system de...,0.196323
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...,skills â r â python â sap hana â tableau â sap...,skills r python sap hana tableau sap hana sql ...,0.292168
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab...",education details mca ymcaust faridabad haryan...,education details mca ymcaust faridabad haryan...,0.181914
5,Data Science,"SKILLS C Basics, IOT, Python, MATLAB, Data Sci...",skills c basics iot python matlab data science...,skills c basics iot python matlab data science...,0.249574
6,Data Science,Skills â¢ Python â¢ Tableau â¢ Data Visuali...,skills â python â tableau â data visualization...,skills python tableau data visualization r stu...,0.491361
7,Data Science,Education Details \r\n B.Tech Rayat and Bahr...,education details b tech rayat bahra institute...,education details rayat bahra institute engine...,0.348964
8,Data Science,Personal Skills â¢ Ability to quickly grasp t...,personal skills â ability quickly grasp techni...,personal skills ability quickly grasp technica...,0.373171
9,Data Science,Expertise â Data and Quantitative Analysis â...,expertise â data quantitative analysis â decis...,expertise data quantitative analysis decision ...,0.609800


In [139]:
df.tail(50)

,Category,Resume,processed_text,processed_text1,similarity_score
912,Testing,Skill Set OS Windows XP/7/8/8.1/10 Database MY...,skill set os windows xp 7 8 8 1 10 database my...,skill set os windows database mysql sql server...,0.054961
913,Testing,â¢ Good logical and analytical skills â¢ Pos...,â good logical analytical skills â positive at...,good logical analytical skills positive attitu...,0.034893
914,Testing,COMPUTER PROFICIENCY â¢ Basic: MS-Office (Pow...,computer proficiency â basic ms office powerpo...,computer proficiency basic powerpoint word out...,0.032685
915,Testing,Computer Skills: â¢ Proficient in MS office (...,computer skills â proficient ms office word ba...,computer skills proficient ms office word basi...,0.034031
916,Testing,â Willingness to accept the challenges. â ...,â willingness accept challenges â positive thi...,willingness accept challenges positive thinkin...,0.152754
917,Testing,"PERSONAL SKILLS â¢ Quick learner, â¢ Eagerne...",personal skills â quick learner â eagerness le...,personal skills quick learner eagerness learn ...,0.073119
918,Testing,COMPUTER SKILLS & SOFTWARE KNOWLEDGE MS-Power ...,computer skills software knowledge ms power po...,computer skills software knowledge point ms of...,0.063829
919,Testing,Skill Set OS Windows XP/7/8/8.1/10 Database MY...,skill set os windows xp 7 8 8 1 10 database my...,skill set os windows database mysql sql server...,0.054961
920,Testing,â¢ Good logical and analytical skills â¢ Pos...,â good logical analytical skills â positive at...,good logical analytical skills positive attitu...,0.034893
921,Testing,COMPUTER PROFICIENCY â¢ Basic: MS-Office (Pow...,computer proficiency â basic ms office powerpo...,computer proficiency basic powerpoint word out...,0.032685


In [203]:
jobs_df.head()

,Job Id,Experience,Qualifications,Salary Range,location,Country,latitude,longitude,Work Type,Company Size,...,Contact,Job Title,Role,Job Portal,Job Description,Benefits,skills,Responsibilities,Company,Company Profile
0,1089843540111562,5 to 15 Years,M.Tech,$59K-$99K,Douglas,Isle of Man,54.2361,-4.5481,Intern,26801,...,001-381-930-7517x737,Digital Marketing Specialist,Social Media Manager,Snagajob,Social Media Managers oversee an organizations...,"{'Flexible Spending Accounts (FSAs), Relocatio...","Social media platforms (e.g., Facebook, Twitte...","Manage and grow social media accounts, create ...",Icahn Enterprises,"{""Sector"":""Diversified"",""Industry"":""Diversifie..."
1,398454096642776,2 to 12 Years,BCA,$56K-$116K,Ashgabat,Turkmenistan,38.9697,59.5563,Intern,100340,...,461-509-4216,Web Developer,Frontend Web Developer,Idealist,Frontend Web Developers design and implement u...,"{'Health Insurance, Retirement Plans, Paid Tim...","HTML, CSS, JavaScript Frontend frameworks (e.g...","Design and code user interfaces for websites, ...",PNC Financial Services Group,"{""Sector"":""Financial Services"",""Industry"":""Com..."
2,481640072963533,0 to 12 Years,PhD,$61K-$104K,Macao,"Macao SAR, China",22.1987,113.5439,Temporary,84525,...,9687619505,Operations Manager,Quality Control Manager,Jobs2Careers,Quality Control Managers establish and enforce...,"{'Legal Assistance, Bonuses and Incentive Prog...",Quality control processes and methodologies St...,Establish and enforce quality control standard...,United Services Automobile Assn.,"{""Sector"":""Insurance"",""Industry"":""Insurance: P..."
3,688192671473044,4 to 11 Years,PhD,$65K-$91K,Porto-Novo,Benin,9.3077,2.3158,Full-Time,129896,...,+1-820-643-5431x47576,Network Engineer,Wireless Network Engineer,FlexJobs,"Wireless Network Engineers design, implement, ...","{'Transportation Benefits, Professional Develo...",Wireless network design and architecture Wi-Fi...,"Design, configure, and optimize wireless netwo...",Hess,"{""Sector"":""Energy"",""Industry"":""Mining, Crude-O..."
4,117057806156508,1 to 12 Years,MBA,$64K-$87K,Santiago,Chile,-35.6751,-71.5429,Intern,53944,...,343.975.4702x9340,Event Manager,Conference Manager,Jobs2Careers,A Conference Manager coordinates and manages c...,"{'Flexible Spending Accounts (FSAs), Relocatio...",Event planning Conference logistics Budget man...,Specialize in conference and convention planni...,Cairn Energy,"{""Sector"":""Energy"",""Industry"":""Energy - Oil & ..."


In [202]:
df.head()

,Category,Resume
0,Data Science,Skills * Programming Languages: Python (pandas...
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...
2,Data Science,"Areas of Interest Deep Learning, Control Syste..."
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab..."


In [204]:
# Clean and preprocess text data
df['processes_test'] = df['Resume'].apply(preprocess_text)
jobs_df['processes_test'] = jobs_df['Job Description'].apply(preprocess_text)

In [205]:
# Create TF-IDF matrix
tfidf = TfidfVectorizer()
resumes_tfidf = tfidf.fit_transform(df['processes_test'])
jobs_tfidf = tfidf.transform(jobs_df['processes_test'])

In [ ]:
# 

In [206]:
# Create label indicating if resume matches job
df['match'] = df.index.map(lambda x: x in jobs_df.index)
# df['match'] = df['processed_text'].index.map(lambda x: x in jobs_df['processes_test'].index)

In [207]:
df.head()

,Category,Resume,processes_test,match
0,Data Science,Skills * Programming Languages: Python (pandas...,skills programming languages python pandas num...,True
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...,education details may 2013 may 2017 b e uit rg...,True
2,Data Science,"Areas of Interest Deep Learning, Control Syste...",areas interest deep learning control system de...,True
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...,skills â r â python â sap hana â tableau â sap...,True
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab...",education details mca ymcaust faridabad haryan...,True


In [210]:
df['match'].unique()

array([ True])

In [211]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(resumes_tfidf, df[['match']], test_size=0.2, random_state=42)

In [212]:
X_train.toarray().shape

(769, 7484)

In [213]:
X_test.toarray().shape

(193, 7484)

In [214]:
y_train.shape

(769, 1)

In [215]:
y_test.shape

(193, 1)

In [195]:
# Check unique classes in your target variable
print(np.unique(y_train))

[ True]


In [196]:
# y_train = y_train.ravel()  # or y_train.flatten() , ravel
# y_train

In [216]:
# Example of potential fix
from sklearn.preprocessing import LabelEncoder

In [217]:
# If using binary classification, ensure you have both True and False
le = LabelEncoder()
y_train = le.fit_transform(y_train)

C:\Users\DELL\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [218]:
y_train

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [180]:
# # Or explicitly convert to binary
# y_train = (y_train > threshold).astype(int)  # replace threshold with appropriate value

In [219]:
# Train logistic regression model
#model = LogisticRegression()
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [221]:
# Evaluate model performance
y_pred = model.predict(X_test)

In [222]:
accuracy_score(y_test, y_pred)*100

0.0

In [ ]:
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
print(f"Resume Screening Model Performance:")
print(f"Accuracy: {accuracy:.2f}, Precision: {precision:.2f}, Recall: {recall:.2f}, F1-score: {f1:.2f}")